Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.

##### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Intel® Gaudi® Accelerator Using Hugging Face Transformer Reinforcement Learning


This document provides instructions on setting up the Intel Gaudi 2 AI accelerator Instance on the Intel® Developer Cloud or any on-premise Intel Gaudi Node. You will be running models from the Intel Gaudi software Model References and the Hugging Face Optimum Habana library.

This assumes that you have setup the latest Intel Gaudi PyTorch Docker image.

The first step is to install the Optimum Habana repository from GitHub and run the demo of Transformer Reinforcement Learning.

### Fine-tuning with Hugging Face Optimum Habana Library
The Optimum Habana library is the interface between the Hugging Face Transformers and Diffusers libraries and the Gaudi 2 card. It provides a set of tools enabling easy model loading, training and inference on single and multi-card settings for different downstream tasks. The following example use the DPO and PPO pipeline to fine-tune a Llama 2 7B model.  For more details, see the [TRL](https://github.com/huggingface/optimum-habana/tree/main/examples/trl) examples at the Optimum-Habana GitHub page. 

Follow the below steps to install the stable release from the Optimum Habana examples and library:

1. Clone the Optimum-Habana project and check out the lastest stable release.  This repository gives access to the examples that are optimized for Intel Gaudi:

In [ ]:
%cd ~
!git clone -b v1.15.0 https://github.com/huggingface/optimum-habana.git

2. Install Optimum-Habana library. This will install the latest stable library:

In [ ]:
!pip install optimum-habana==1.15.0

3. In order to use the DeepSpeed library on Intel Gaudi 2, install the Intel Gaudi DeepSpeed fork:

In [ ]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.19.0

The following example is based on the Optimum-Habana TRL task example. Change to the trl directory and install the additional SW requirements for this specific example:

In [ ]:
%cd ~/optimum-habana/examples/trl/
!pip install -U -r requirements.txt

In [ ]:
!huggingface-cli login --token <your_token_here>

### DPO Pipeline

#### Training

The following example is for the creation of StackLlaMa 2: a Stack exchange llama-v2-7b model. There are two main steps to the DPO training process:

1. Supervised fine-tuning of the base llama-v2-7b model to create llama-v2-7b-se:

In [4]:
!python sft.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --dataset_name "lvwerra/stack-exchange-paired" \
    --output_dir="./sft" \
    --max_steps=500 \
    --logging_steps=10 \
    --save_steps=100 \
    --do_train \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=1 \
    --gradient_accumulation_steps=2 \
    --learning_rate=1e-4 \
    --lr_scheduler_type="cosine" \
    --warmup_steps=100 \
    --weight_decay=0.05 \
    --optim="paged_adamw_32bit" \
    --lora_target_modules "q_proj" "v_proj" \
    --bf16 \
    --remove_unused_columns=False \
    --run_name="sft_llama2" \
    --report_to=none \
    --use_habana \
    --use_lazy_mode

/usr/local/lib/python3.10/dist-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoders.vq_model import VQEncoderOutput`, instead.
  deprecate("VQEncoderOutput", "0.31", deprecation_message)
/usr/local/lib/python3.10/dist-packages/diffusers/models/vq_model.py:25: FutureWarning: `VQModel` is deprecated and will be removed in version 0.31. Importing `VQModel` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoders.vq_model import VQModel`, instead.
  deprecate("VQModel", "0.31", deprecation_message)
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, type

To merge the adaptors to get the final sft merged checkpoint, we can use the merge_peft_adapter.py helper script that comes with TRL:

In [5]:
!python merge_peft_adapter.py --base_model_name="meta-llama/Llama-2-7b-hf" --adapter_model_name="sft" --output_name="sft/final_merged_checkpoint"

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.88it/s]
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/hpu/__init__.py:159: UserWarning: torch.hpu.setDeterministic is deprecated and will be removed in next release. Please use torch.use_deterministic_algorithms instead.
  warnings.warn(
[2024-10-19 01:59:45,238] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to hpu (auto detect)


2. Run the DPO trainer using the model saved by the previous step:

In [6]:
!python dpo.py \
    --model_name_or_path="sft/final_merged_checkpoint" \
    --tokenizer_name_or_path=meta-llama/Llama-2-7b-hf \
    --lora_target_modules "q_proj" "v_proj" "k_proj" "out_proj" "fc_in" "fc_out" "wte" \
    --output_dir="dpo" \
    --report_to=none

***** train metrics *****
  epoch                       =     0.0024
  max_memory_allocated (GB)   =      86.99
  memory_allocated (GB)       =      60.24
  total_flos                  =        0GF
  total_memory_available (GB) =      94.62
  train_loss                  =     1.4503
  train_runtime               = 0:34:31.45
  train_samples_per_second    =      1.931
  train_steps_per_second      =      0.483


#### Merging the adaptors

To merge the adaptors into the base model we can use the merge_peft_adapter.py helper script that comes with TRL:

In [7]:
!python merge_peft_adapter.py --base_model_name="meta-llama/Llama-2-7b-hf" --adapter_model_name="dpo" --output_name="stack-llama-2"

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.84it/s]
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)
/usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/hpu/__init__.py:159: UserWarning: torch.hpu.setDeterministic is deprecated and will be removed in next release. Please use torch.use_deterministic_algorithms instead.
  warnings.warn(
[2024-10-19 03:46:58,894] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to hpu (auto detect)


which will also push the model to your HuggingFace hub account.

#### Running the model

We can load the DPO-trained LoRA adaptors which were saved by the DPO training step and run it through the [text-generation example]([../text-generation/](https://github.com/huggingface/optimum-habana/tree/main/examples/text-generation)).

In [8]:
%!python ~/optimum-habana/examples/text-generation/run_generation.py \
--model_name_or_path stack-llama-2/ \
--use_hpu_graphs --use_kv_cache --batch_size 1 --bf16 --do_sample --max_new_tokens 50 \
--temperature 0.5 \
--top_p 0.5 \
--prompt "When I go to New York I always go see "

/root/optimum-habana/examples/text-generation
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
10/19/2024 03:47:19 - INFO - __main__ - Single-device run.
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:00<00:00,  6.78it/s]
============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 1
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
 PT_HPU_EAGER_PIPELINE_ENABLE = 1
 PT_HPU_EAG

## Next Steps
You now have access to all the Models in Model-References and Optimum-Habana repositories, you can start to look at other models.  Remember that all the models in these repositories are fully documented so they are easy to use.
* To explore more models from the Model References, start [here](https://github.com/HabanaAI/Model-References).  
* To run more examples using Hugging Face go [here](https://github.com/huggingface/optimum-habana?tab=readme-ov-file#validated-models).  
* To migrate other models to Gaudi 2, refer to PyTorch Model Porting in the [documentation](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html)

In [ ]:
exit()